In [ ]:
import pandas as pd

pd.options.plotting.backend = "plotly"
from pyts.decomposition import SingularSpectrumAnalysis
import plotly.express as px
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
import plotly.express as px
from statsmodels.tsa.seasonal import seasonal_decompose
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from scipy import signal

## Dataset: Energy consumption
These data has been extracted from *RTE* and *MeteoFrance* APIs
### Columns : 
- **Consommation** : Britanny power consumption in MW
- **Température (°C)** : External temperature
- **Nebulosité totale** : Cloud cover
- **Date**

In [ ]:
# Load dataset
DATAPATH = "../Dataset/Consommation_Bretagne.csv"
df = pd.read_csv(DATAPATH)
df["Date"] = pd.to_datetime(df["Date"])
df = df.set_index("Date")

In [ ]:
fig = df.plot()
fig.update_layout(width=800, height=600)
fig.show()

## Analyses

### ACF/PACF

In [ ]:
import statsmodels.api as sm
import plotly.subplots as sp

# Calculate ACF and PACF values
acf_values = sm.tsa.acf(df["Consommation"], nlags=24 * 7 * 2)
pacf_values = sm.tsa.pacf(df["Consommation"], nlags=24 * 7 * 2)

# Create a subplot with two subplots (one for ACF and one for PACF)
fig = sp.make_subplots(
    rows=1,
    cols=2,
    subplot_titles=[
        "Autocorrelation Function (ACF)",
        "Partial Autocorrelation Function (PACF)",
    ],
)

# Add ACF trace
acf_trace = go.Scatter(
    x=list(range(len(acf_values))), y=acf_values, mode="lines", name="ACF"
)
fig.add_trace(acf_trace, row=1, col=1)

# Add PACF trace
pacf_trace = go.Scatter(
    x=list(range(len(pacf_values))), y=pacf_values, mode="lines", name="PACF"
)
fig.add_trace(pacf_trace, row=1, col=2)

# Update layout
fig.update_layout(title_text="ACF and PACF Plots", showlegend=True)

# Show the figure
fig.show()

### Spectral analyses

In [ ]:
time_series = df["Consommation"].values
time = np.arange(len(time_series))

# Compute the FFT
fft_result = np.fft.fft(time_series)

# Calculate Frequencies
n = len(time_series)
timestep = 1  # Set your time step here (e.g., 1 day, 1 hour)
freq = np.fft.fftfreq(n, d=timestep)

# Keep only positive frequencies
positive_freqs = freq > 0
fft_positive = fft_result[positive_freqs]
freq_positive = freq[positive_freqs]

# Calculate Power Spectral Density for positive frequencies
psd_positive = np.abs(fft_positive) ** 2 / n

fig = make_subplots(
    rows=1, cols=2, subplot_titles=("Time Series", "Power Spectral Density")
)
fig.add_trace(
    go.Scatter(x=time, y=time_series, mode="lines", name="Time Series"), row=1, col=1
)
fig.add_trace(
    go.Scatter(x=freq_positive, y=psd_positive, mode="lines", name="PSD"), row=1, col=2
)

fig.update_layout(
    title="Time Series and its Power Spectral Density",
    xaxis_title="Time",
    xaxis2_title="Frequency",
    yaxis_title="Amplitude",
    yaxis2_title="Power",
)
fig.show()

In [ ]:
# Sample time series data
time_series = df["Consommation"].values
time = np.arange(len(time_series))
sampling_rate = 1  # Define the sampling rate of your time series

# Define window length (nperseg) and overlap (noverlap)
nperseg = 24 * 7 * 4 * 2  # Define the window length as per your analysis requirement
noverlap = nperseg - 1  # For a stride of 1

# Compute the Spectrogram
frequencies, times, spectrogram = signal.spectrogram(
    time_series, fs=sampling_rate, nperseg=nperseg, noverlap=noverlap
)

fig = make_subplots(
    rows=1,
    cols=2,
    subplot_titles=("Time Series", "Spectrogram"),
    specs=[[{"type": "scatter"}, {"type": "heatmap"}]],
)
fig.add_trace(
    go.Scatter(x=time, y=time_series, mode="lines", name="Time Series"), row=1, col=1
)
fig.add_trace(
    go.Heatmap(
        x=times,
        y=frequencies,
        z=10 * np.log10(spectrogram),
        colorscale="Jet",
        colorbar=dict(title="Power dB", x=0.46),
        name="Spectrogram",
    ),
    row=1,
    col=2,
)
fig.update_layout(
    title="Time Series and its Spectrogram",
    xaxis_title="Time",
    xaxis2_title="Time",
    yaxis_title="Amplitude",
    yaxis2_title="Frequency",
)
fig.show()

### Calendar informations

In [ ]:
df["Weekday"] = df.index.weekday
df["Hour"] = df.index.hour
df["Month"] = df.index.month

In [ ]:
weekday_hour_mat = (
    df.groupby(["Weekday", "Hour"], as_index=False)
    .mean()[["Weekday", "Hour", "Consommation"]]
    .pivot(index="Weekday", columns="Hour", values="Consommation")
)
fig = px.imshow(
    weekday_hour_mat,
    labels=dict(x="Hour", y="Weekday", color="Consommation"),
    x=weekday_hour_mat.columns,
    y=weekday_hour_mat.index,
    color_continuous_scale="Viridis",
    title="Consumption Heatmap",
)
fig.update_layout(width=800, height=600)
fig.show()

In [ ]:
month_weekday_mat = (
    df.groupby(["Weekday", "Month"], as_index=False)
    .mean()[["Weekday", "Month", "Consommation"]]
    .pivot(index="Weekday", columns="Month", values="Consommation")
)
fig = px.imshow(
    month_weekday_mat,
    labels=dict(x="Month", y="Weekday", color="Consommation"),
    x=month_weekday_mat.columns,
    y=month_weekday_mat.index,
    color_continuous_scale="Viridis",
    title="Consumption Heatmap",
)
fig.update_layout(width=800, height=600)
fig.show()

### Exogenous features

In [ ]:
temperature_weekday_mat = df.copy()
temperature_weekday_mat["Température (°C)"] = temperature_weekday_mat[
    "Température (°C)"
].astype(int)
fig = temperature_weekday_mat.groupby("Température (°C)").mean()["Consommation"].plot()
fig.update_layout(width=800, height=600)
fig.show()

In [ ]:
fig = (
    df[["Température (°C)", "Consommation"]]
    .rolling(24 * 7)
    .corr()[["Consommation"]]
    .unstack(level=1)["Consommation"]["Température (°C)"]
    .rolling(24 * 7, step=24 * 7)
    .mean()
    .plot()
)
fig.update_layout(width=800, height=600)
fig.show()

### Decomposition

In [ ]:
# Applying seasonal decomposition to the daily resampled data
# Decomposition into trend, seasonal, and residual components
decomposition = seasonal_decompose(
    df["Consommation"].resample("D").sum(), model="additive"
)

# Extracting the trend, seasonal, and residual components
trend = decomposition.trend.dropna()
seasonal = decomposition.seasonal.dropna()
residual = decomposition.resid.dropna()

# Plotting the decomposition components using Plotly
fig_decomposed = go.Figure()
fig_decomposed.add_trace(go.Scatter(x=trend.index, y=trend, mode="lines", name="Trend"))
fig_decomposed.add_trace(
    go.Scatter(x=seasonal.index, y=seasonal, mode="lines", name="Seasonal")
)
fig_decomposed.add_trace(
    go.Scatter(x=residual.index, y=residual, mode="lines", name="Residual")
)

fig_decomposed.update_layout(
    title="Seasonal Decomposition of Daily Time Series",
    xaxis_title="Time",
    yaxis_title="Value",
    template="plotly_white",
)
fig_decomposed.update_layout(width=800, height=600)
fig_decomposed.show()

In [ ]:
# Resample the data to daily frequency, averaging the values of each day
daily_data = df.resample("D").mean()

# Feature Engineering: Adding calendar features and lagged variables
daily_data["weekday"] = daily_data.index.weekday
daily_data["month"] = daily_data.index.month

# Adding lagged variables for up to 7 days for temperature and consumption
for i in range(1, 8):
    daily_data[f"lagged_temp_{i}"] = daily_data["Température (°C)"].shift(i)
    daily_data[f"lagged_consumption_{i}"] = daily_data["Consommation"].shift(i)

# Drop rows with NaN values resulting from lagged features creation
daily_data = daily_data.dropna()

# Define the feature set for the model
feature_columns = (
    ["Température (°C)", "Nebulosité totale", "weekday", "month"]
    + [f"lagged_temp_{i}" for i in range(1, 8)]
    + [f"lagged_consumption_{i}" for i in range(1, 8)]
)
X = daily_data[feature_columns]
y = daily_data["Consommation"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=False
)

# Initialize and train the Random Forest model
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Predict power consumption on the testing set
y_pred = model.predict(X_test)

# Calculate performance metrics
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)

# Prepare Plotly visualizations for the decomposed components
fig_original = px.line(daily_data["Consommation"], title="Original Power Consumption")
fig_trend = px.line(decomposition.trend, title="Trend Component of Power Consumption")
fig_seasonal = px.line(
    decomposition.seasonal, title="Seasonal Component of Power Consumption"
)
fig_residual = px.line(decomposition.resid, title="Residuals of Power Consumption")

# Display performance metrics
print(f"RMSE: {rmse}")
print(f"R^2: {r2}")
print(f"MAPE: {mape}")